In [1]:

%pprint
%run ../load_magic/soup.py

tables_url = 'https://en.wikipedia.org/wiki/List_of_natural_satellites'
table_dfs_list = get_wiki_tables(tables_url, verbose=True)

Pretty printing has been turned OFF
[(6, (223, 12)), (8, (4, 10)), (5, (9, 3)), (2, (1, 17)), (9, (5, 3)), (1, (1, 9)), (0, (8, 1)), (4, (1, 2)), (7, (1, 2)), (3, (1, 1))]


In [5]:

moons_df = table_dfs_list[6].copy()
moons_df.columns = ['_'.join(re.sub('[^A-Za-z0-9]+', ' ', cn).strip().split(' ')).lower() for cn in moons_df.columns]
moons_df.mean_radius_km = moons_df.mean_radius_km.map(lambda x: float(re.sub(r'[^\.0-9]+', '', str(x).split('±')[0])))
parents_list = ['Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune']
mask_series = moons_df.parent.isin(parents_list)
columns_list = ['name', 'parent', 'mean_radius_km']
moons_df[mask_series][columns_list]

,name,parent,mean_radius_km
0,Moon,Earth,1738.000
1,Phobos,Mars,11.267
2,Deimos,Mars,6.200
3,Io,Jupiter,1821.600
4,Europa,Jupiter,1560.800
...,...,...,...
202,Psamathe,Neptune,20.000
203,Sao,Neptune,22.000
204,Laomedeia,Neptune,21.000
205,Neso,Neptune,30.000


In [7]:

tables_url = 'https://en.wikipedia.org/wiki/List_of_gravitationally_rounded_objects_of_the_Solar_System'
table_dfs_list = get_wiki_tables(tables_url, verbose=True)

[(8, (21, 12)), (3, (25, 10)), (9, (20, 12)), (5, (25, 7)), (6, (22, 7)), (1, (21, 3)), (10, (12, 2)), (11, (9, 2)), (0, (8, 1)), (12, (4, 2)), (13, (4, 2)), (7, (1, 7)), (4, (1, 6)), (2, (1, 3))]


In [8]:

planets_df = table_dfs_list[3].copy()
planet_regex = re.compile('([A-z][a-z]+)')
columns_list = []
for x in planets_df.columns:
    if 'Unnamed' in x:
        planet_name = x
    else:
        match_obj = planet_regex.search(str(x))
        if match_obj:
            planet_name = match_obj.group(1)
    columns_list.append(planet_name)
planets_df.columns = columns_list
columns_list = planets_df.columns[:2]
new_columns = []
for values_list in planets_df[columns_list].values:
    values_list = [str(x) for x in values_list]
    if values_list[0] == values_list[1]:
        new_columns.append(values_list[0])
    else:
        new_columns.append(' '.join(values_list))
planets_df = planets_df.T.iloc[2:]
planets_df.columns = new_columns
planets_df = planets_df.T.iloc[2:].T
planets_df.columns = ['_'.join(re.sub('[^A-Za-z0-9]+', ' ', cn).strip().split(' ')).lower() for cn in planets_df.columns]
planets_df['equatorial_radius_km'] = planets_df.equatorial_radius_km_e_f.map(lambda x: float(str(x).split(' ')[0].replace(',', '')))
columns_list = ['equatorial_radius_km_e_f', 'equatorial_radius_km']
planets_df[columns_list]

,equatorial_radius_km_e_f,equatorial_radius_km
Mercury,"2,440.53 0.3826",2440.5300
Venus,"6,051.8 0.9488",6051.8000
Earth,"6,378.1366 1",6378.1366
Mars,"3,396.19 0.53247",3396.1900
Jupiter,"71,492 11.209",71492.0000
Saturn,"60,268 9.449",60268.0000
Uranus,"25,559 4.007",25559.0000
Neptune,"24,764 3.883",24764.0000


In [16]:

radius_dict = planets_df.equatorial_radius_km.to_dict()
def f(row_series):
    parent = row_series.parent
    parent_radius_km = radius_dict.get(parent, np.nan)
    moon_radius_km = row_series.mean_radius_km
    parent_radius_ratio = parent_radius_km / moon_radius_km
    
    return parent_radius_ratio
moons_df['parent_radius_ratio'] = moons_df.apply(f, axis='columns')
columns_list = ['name', 'parent', 'mean_radius_km', 'parent_radius_ratio']
moons_df[mask_series][columns_list].sort_values('parent_radius_ratio').head()

,name,parent,mean_radius_km,parent_radius_ratio
0,Moon,Earth,1738.00,3.669814
193,Triton,Neptune,1353.40,18.297621
88,Titan,Saturn,2574.73,23.407503
5,Ganymede,Jupiter,2634.10,27.140959
6,Callisto,Jupiter,2410.30,29.661038


In [17]:

%run ../load_magic/storage.py

s = Storage()
s.store_objects(moons_df=moons_df, planets_df=planets_df)

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\notebooks\saves\pkl\moons_df.pkl
Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\notebooks\saves\pkl\planets_df.pkl
